# 🚀 Pipeline Heston Complet dans un Notebook

Ce notebook reprend la logique de `streamlit_app.py` et déroule les étapes :
1. Téléchargement des données d'options via Yahoo Finance.
2. Calibration des paramètres Heston par réseau de neurones PyTorch.
3. Calcul de surfaces d'IV analytiques par Carr-Madan puis inversion Black-Scholes.
4. Pricing Monte Carlo Heston et comparaison vs les prix analytiques.

Exécutez les cellules séquentiellement pour reproduire l'analyse complète.

## Pré-requis
- Installer les dépendances : `pip install -r requirements.txt`
- Une clé réseau active pour que `yfinance` récupère les options en direct.
- GPU facultatif (le script tourne sur CPU).

In [67]:
from __future__ import annotations

import math
import requests
import re
from pathlib import Path
from typing import Callable, Dict, Tuple

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch
import yfinance as yf
from IPython.display import display
from scipy.interpolate import griddata

from heston_torch import HestonParams, carr_madan_call_torch

torch.set_default_dtype(torch.float64)
DEVICE = torch.device("cpu")
MIN_IV_MATURITY = 0.1

## Fonctions utilitaires : marché et simulation Monte Carlo

In [68]:
def heston_mc_pricer(
    S0: float,
    K: float,
    T: float,
    r: float,
    v0: float,
    theta: float,
    kappa: float,
    sigma_v: float,
    rho: float,
    n_paths: int = 50_000,
    n_steps: int = 100,
    option_type: str = "call",
) -> float:
    """Pricer Monte Carlo pour options européennes sous Heston."""
    dt = T / n_steps
    sqrt_dt = math.sqrt(dt)

    S = np.full(n_paths, S0)
    v = np.full(n_paths, v0)

    for _ in range(n_steps):
        Z1 = np.random.randn(n_paths)
        Z2 = np.random.randn(n_paths)
        Z_S = Z1
        Z_v = rho * Z1 + math.sqrt(1 - rho**2) * Z2

        S = S * np.exp((r - 0.5 * np.maximum(v, 0)) * dt + np.sqrt(np.maximum(v, 0)) * sqrt_dt * Z_S)
        v = v + kappa * (theta - np.maximum(v, 0)) * dt + sigma_v * np.sqrt(np.maximum(v, 0)) * sqrt_dt * Z_v
        v = np.maximum(v, 0)

    payoff = np.maximum(S - K, 0) if option_type == "call" else np.maximum(K - S, 0)
    return math.exp(-r * T) * np.mean(payoff)


def fetch_spot(symbol: str) -> float:
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period="1d")
    if hist.empty:
        raise RuntimeError("Unable to retrieve spot price.")
    return float(hist["Close"].iloc[-1])


def _select_monthly_expirations(expirations, years_ahead: float = 2.5) -> list[str]:
    today = pd.Timestamp.utcnow().date()
    limit_date = today + pd.Timedelta(days=365 * years_ahead)
    monthly: Dict[Tuple[int, int], Tuple[pd.Timestamp, str]] = {}
    for exp in expirations:
        exp_ts = pd.Timestamp(exp)
        exp_date = exp_ts.date()
        if not (today < exp_date <= limit_date):
            continue
        key = (exp_date.year, exp_date.month)
        if key not in monthly or exp_ts < monthly[key][0]:
            monthly[key] = (exp_ts, exp)
    return [item[1] for item in sorted(monthly.values(), key=lambda x: x[0])]


def download_options(symbol: str, option_type: str, years_ahead: float = 2.5, save_csv: bool = True) -> pd.DataFrame:
    ticker = yf.Ticker(symbol)
    spot = fetch_spot(symbol)
    expirations = ticker.options
    if not expirations:
        raise RuntimeError(f"No option expirations found for {symbol}")
    selected = _select_monthly_expirations(expirations, years_ahead)
    rows: list[dict] = []
    now = pd.Timestamp.utcnow().tz_localize(None)
    for expiry in selected:
        expiry_dt = pd.Timestamp(expiry)
        T = max((expiry_dt - now).total_seconds() / (365.0 * 24 * 3600), 0.0)
        chain = ticker.option_chain(expiry)
        data = chain.calls if option_type == "call" else chain.puts
        price_col = "C_mkt" if option_type == "call" else "P_mkt"
        for _, row in data.iterrows():
            rows.append(
                {
                    "S0": spot,
                    "K": float(row["strike"]),
                    "T": T,
                    price_col: float(row["lastPrice"]),
                    "iv_market": float(row.get("impliedVolatility", float("nan"))),
                }
            )
    df = pd.DataFrame(rows)
    if save_csv:
        out_dir = Path("data")
        out_dir.mkdir(parents=True, exist_ok=True)
        ts = pd.Timestamp.utcnow().strftime("%Y%m%d_%H%M%S")
        out_path = out_dir / f"{symbol}_{option_type}_options_{ts}.csv"
        df.to_csv(out_path, index=False)
        print(f"Options {option_type} sauvegardées dans {out_path}")
    return df

## Calibration Heston NN et fonctions Black-Scholes

In [69]:
def prices_from_unconstrained(
    u: torch.Tensor,
    S0_t: torch.Tensor,
    K_t: torch.Tensor,
    T_t: torch.Tensor,
    r: float,
    q: float,
) -> torch.Tensor:
    params = HestonParams.from_unconstrained(u[0], u[1], u[2], u[3], u[4])
    prices = []
    for S0_i, K_i, T_i in zip(S0_t, K_t, T_t):
        price_i = carr_madan_call_torch(S0_i, r, q, T_i, params, K_i)
        prices.append(price_i)
    return torch.stack(prices)


def loss(
    u: torch.Tensor,
    S0_t: torch.Tensor,
    K_t: torch.Tensor,
    T_t: torch.Tensor,
    C_mkt_t: torch.Tensor,
    r: float,
    q: float,
) -> torch.Tensor:
    model_prices = prices_from_unconstrained(u, S0_t, K_t, T_t, r, q)
    diff = model_prices - C_mkt_t
    return 0.5 * (diff**2).mean()


def calibrate_heston_nn(
    df: pd.DataFrame,
    r: float,
    q: float,
    max_points: int = 1000,
    max_iters: int = 200,
    lr: float = 5e-3,
    progress_callback: Callable[[int, int], None] | None = None,
    log_callback: Callable[[str], None] | None = None,
    display_every: int = 10,
) -> dict:
    if df.empty:
        raise ValueError("DataFrame vide.")

    df_clean = df.dropna(subset=["S0", "K", "T", "C_mkt"])
    df_clean = df_clean[df_clean["T"] > 0]
    df_clean = df_clean[df_clean["C_mkt"] > 0]

    if len(df_clean) == 0:
        raise ValueError("Aucun point valide après nettoyage.")

    if len(df_clean) > max_points:
        df_clean = df_clean.sample(n=max_points, random_state=42)

    S0_t = torch.tensor(df_clean["S0"].values, dtype=torch.float64, device=DEVICE)
    K_t = torch.tensor(df_clean["K"].values, dtype=torch.float64, device=DEVICE)
    T_t = torch.tensor(df_clean["T"].values, dtype=torch.float64, device=DEVICE)
    C_mkt_t = torch.tensor(df_clean["C_mkt"].values, dtype=torch.float64, device=DEVICE)

    u = torch.zeros(5, dtype=torch.float64, device=DEVICE, requires_grad=True)
    optimizer = torch.optim.Adam([u], lr=lr)

    for iteration in range(max_iters):
        optimizer.zero_grad()
        loss_val = loss(u, S0_t, K_t, T_t, C_mkt_t, r, q)
        loss_val.backward()
        optimizer.step()

        if progress_callback:
            progress_callback(iteration + 1, max_iters)
        if log_callback:
            log_callback(f"Iter {iteration + 1}/{max_iters} | Loss = {loss_val.item():.6f}")
        elif (iteration + 1) % display_every == 0 or iteration == 0:
            print(f"Iter {iteration + 1}/{max_iters} | Loss = {loss_val.item():.6f}")

    params = HestonParams.from_unconstrained(u[0], u[1], u[2], u[3], u[4])

    return {
        "kappa": float(params.kappa.cpu().detach()),
        "theta": float(params.theta.cpu().detach()),
        "sigma": float(params.sigma.cpu().detach()),
        "rho": float(params.rho.cpu().detach()),
        "v0": float(params.v0.cpu().detach()),
    }


def bs_call(S: float, K: float, T: float, r: float, sigma: float) -> float:
    if T <= 0 or sigma <= 0:
        return max(S - K, 0)
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    from scipy.stats import norm
    return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)


def bs_put(S: float, K: float, T: float, r: float, sigma: float) -> float:
    if T <= 0 or sigma <= 0:
        return max(K - S, 0)
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    from scipy.stats import norm
    return K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)


def implied_vol_option(
    price: float,
    S: float,
    K: float,
    T: float,
    r: float,
    option_type: str = "call",
    tol: float = 1e-6,
    max_iter: int = 100,
) -> float:
    if T < MIN_IV_MATURITY:
        return np.nan

    intrinsic = max(S - K, 0) if option_type == "call" else max(K - S, 0)
    if price <= intrinsic:
        return np.nan

    sigma = 0.3
    for _ in range(max_iter):
        price_est = bs_call(S, K, T, r, sigma) if option_type == "call" else bs_put(S, K, T, r, sigma)
        diff = price_est - price
        if abs(diff) < tol:
            return sigma

        d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
        from scipy.stats import norm
        vega = S * norm.pdf(d1) * math.sqrt(T)
        if vega < 1e-10:
            return np.nan

        sigma = sigma - diff / vega
        if sigma <= 0:
            return np.nan

    return np.nan

## Paramètres de la session

In [70]:
ticker = "SPY"
rf_rate = 0.02
div_yield = 0.0
years_ahead = 2.5
iv_T_max = 2.0  # Horizon max pour la surface IV (T grid)
calib_T_target = 0.86  # Maturité utilisée pour la calibration ciblée
calib_span = 10  # Bande K autour de S0 pour la calibration
T_mc = 0.86
span_mc = 20.0
span_k_grid = span_mc  # Span utilisé pour filtrer les strikes autour de S0 (K = +/- span)
step_strike = 1.0
n_maturities = 40
n_paths = 5000
max_iters = 1000
max_points = 1_000
learning_rate = 0.005
use_cboe_data = True  # Activer la récupération via CBOE delayed (gratuit)
cboe_save_call = "data/testcall.csv"
cboe_save_put = "data/testput.csv"

print(f"Ticker = {ticker} | r = {rf_rate:.2%} | q = {div_yield:.2%}")

Ticker = SPY | r = 2.00% | q = 0.00%


## Étape 0 — CBOE delayed (gratuit)

Fonction utilitaire pour récupérer la chaîne d'options CBOE (données retardées 15 min) et sauvegarder dans `data/testcall.csv` / `data/testput.csv`.

In [71]:
def download_options_cboe(symbol: str, option_type: str = "call", save_csv: bool = True, out_path: str | None = None):
    # Chaîne d'options via CBOE delayed (gratuite) sans filtre K
    import numpy as np  # import local pour éviter les surprises

    url = f"https://cdn.cboe.com/api/global/delayed_quotes/options/{symbol.upper()}.json"
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    payload = resp.json()
    data = payload.get("data", {})
    options = data.get("options", [])
    spot = float(data.get("current_price") or data.get("close") or np.nan)

    now = pd.Timestamp.utcnow().tz_localize(None)
    pattern = re.compile(rf"^{symbol.upper()}(?P<expiry>\d{{6}})(?P<cp>[CP])(?P<strike>\d+)$")

    rows: list[dict] = []
    for opt in options:
        opt_code = opt.get("option", "")
        match = pattern.match(opt_code)
        if not match:
            continue
        cp = match.group("cp")
        if (option_type == "call" and cp != "C") or (option_type == "put" and cp != "P"):
            continue

        expiry_dt = pd.to_datetime(match.group("expiry"), format="%y%m%d")
        T = (expiry_dt - now).total_seconds() / (365.0 * 24 * 3600)
        if T <= 0:
            continue
        T = round(T, 2)
        if T <= 0:
            continue

        strike = int(match.group("strike")) / 1000.0

        bid = float(opt.get("bid") or 0.0)
        ask = float(opt.get("ask") or 0.0)
        last = float(opt.get("last_trade_price") or 0.0)
        mid = np.nan
        if bid > 0 and ask > 0:
            mid = 0.5 * (bid + ask)
        elif last > 0:
            mid = last
        mid = round(mid, 2)
        if np.isnan(mid) or mid <= 0:
            continue

        iv_val = opt.get("iv", np.nan)
        iv_val = float(iv_val) if iv_val not in (None, "") else np.nan

        rows.append({
            "S0": spot,
            "K": strike,
            "T": T,
            "C_mkt": mid if option_type == "call" else np.nan,
            "P_mkt": mid if option_type == "put" else np.nan,
            "iv_market": iv_val,
        })

    df = pd.DataFrame(rows)
    if save_csv:
        target = Path(out_path or f"data/test{'call' if option_type == 'call' else 'put'}.csv")
        target.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(target, index=False)
        print(f"CBOE {option_type} sauvegardé dans {target} ({len(df)} lignes)")
    return df, spot


## Étape 1 — Téléchargement des options

Deux jeux de données sont construits :
- `calls_df_full` / `puts_df_full` téléchargés (complets).
- `calls_df_train` / `puts_df_train` identiques (pas de filtre K appliqué).

In [72]:
if use_cboe_data:
    print("Mode CBOE delayed activé (gratuit)…")
    calls_df_full, S0_ref = download_options_cboe(ticker, "call", save_csv=True, out_path=cboe_save_call)
    puts_df_full, _ = download_options_cboe(ticker, "put", save_csv=True, out_path=cboe_save_put)
else:
    calls_df_full = download_options(ticker, "call", years_ahead)
    puts_df_full = download_options(ticker, "put", years_ahead)
    S0_ref = fetch_spot(ticker)

# Arrondi des maturités au centième
calls_df_full["T"] = calls_df_full["T"].round(2)
puts_df_full["T"] = puts_df_full["T"].round(2)
# Suppression des maturités nulles
calls_df_full = calls_df_full[calls_df_full["T"] > 0]
puts_df_full = puts_df_full[puts_df_full["T"] > 0]

# Jeux d'entraînement = datasets complets
calls_df_train = calls_df_full.copy()
puts_df_train = puts_df_full.copy()

# Alias utilisés par la suite
calls_df = calls_df_train
puts_df = puts_df_train

print(f"Calls (train complet) : {len(calls_df_train)} | Puts (train complet) : {len(puts_df_train)}")
print(f"Spot S0 = {S0_ref:.2f}")

display(calls_df_train.head())

# Sauvegarde des datasets (train)
calls_df_train.to_csv(cboe_save_call, index=False)
puts_df_train.to_csv(cboe_save_put, index=False)
print(f"Training calls sauvegardés dans {cboe_save_call} ({len(calls_df_train)} lignes)")
print(f"Training puts sauvegardés dans {cboe_save_put} ({len(puts_df_train)} lignes)")


Mode CBOE delayed activé (gratuit)…
CBOE call sauvegardé dans data/testcall.csv (4739 lignes)
CBOE put sauvegardé dans data/testput.csv (4742 lignes)
Calls (train complet) : 4739 | Puts (train complet) : 4742
Spot S0 = 672.97


,S0,K,T,C_mkt,P_mkt,iv_market
0,672.97,500.0,0.01,162.90,NaN,0.0000
1,672.97,505.0,0.01,157.92,NaN,0.7300
2,672.97,510.0,0.01,152.92,NaN,0.6013
3,672.97,515.0,0.01,147.93,NaN,0.6739
4,672.97,520.0,0.01,142.93,NaN,0.6262


Training calls sauvegardés dans data/testcall.csv (4739 lignes)
Training puts sauvegardés dans data/testput.csv (4742 lignes)


## Étape 2 — Calibration Heston NN

In [73]:
def notebook_progress(current: int, total: int) -> None:
    pct = 100 * current / total
    print(f"Progression: {pct:5.1f}% ({current}/{total})")

# Calibration sur la zone d'intérêt (T ≈ calib_T_target, K dans [S0_ref - calib_span, S0_ref + calib_span])
calib_slice = calls_df_train[
    (calls_df_train["T"].round(2) == round(calib_T_target, 2)) &
    (calls_df_train["K"].between(S0_ref - calib_span, S0_ref + calib_span)) &
    (calls_df_train["C_mkt"] > 0.05) &
    (calls_df_train["iv_market"] > 0)
]

print(f"Calibration sur {len(calib_slice)} points (T≈{calib_T_target}, K in [{S0_ref - calib_span:.1f}, {S0_ref + calib_span:.1f}])")
if len(calib_slice) < 5:
    raise ValueError("Pas assez de points pour calibrer dans cette zone.")

calib_params = calibrate_heston_nn(
    calib_slice,
    r=rf_rate,
    q=div_yield,
    max_points=max_points,
    max_iters=max_iters,
    lr=learning_rate,
    progress_callback=notebook_progress,
)

print("Paramètres Heston calibrés:")
display(pd.Series(calib_params, name="valeur"))


Calibration sur 20 points (T≈0.86, K in [663.0, 683.0])
Progression:   0.1% (1/1000)
Iter 1/1000 | Loss = 11673.909665
Progression:   0.2% (2/1000)
Progression:   0.3% (3/1000)
Progression:   0.4% (4/1000)
Progression:   0.5% (5/1000)
Progression:   0.6% (6/1000)
Progression:   0.7% (7/1000)
Progression:   0.8% (8/1000)
Progression:   0.9% (9/1000)
Progression:   1.0% (10/1000)
Iter 10/1000 | Loss = 11121.234418
Progression:   1.1% (11/1000)
Progression:   1.2% (12/1000)
Progression:   1.3% (13/1000)
Progression:   1.4% (14/1000)
Progression:   1.5% (15/1000)
Progression:   1.6% (16/1000)
Progression:   1.7% (17/1000)
Progression:   1.8% (18/1000)
Progression:   1.9% (19/1000)
Progression:   2.0% (20/1000)
Iter 20/1000 | Loss = 10521.046981
Progression:   2.1% (21/1000)
Progression:   2.2% (22/1000)
Progression:   2.3% (23/1000)
Progression:   2.4% (24/1000)
Progression:   2.5% (25/1000)
Progression:   2.6% (26/1000)
Progression:   2.7% (27/1000)
Progression:   2.8% (28/1000)
Progressi

kappa    0.078495
theta    0.276484
sigma    2.560915
rho     -0.762096
v0       0.166482
Name: valeur, dtype: float64

## Étape 3 — Surfaces analytiques Carr-Madan & IV

In [74]:
K_grid = np.arange(S0_ref - span_mc, S0_ref + span_mc + step_strike, step_strike)
T_grid = np.linspace(0.1, min(iv_T_max, years_ahead), n_maturities)
print(f"Grille strikes: {len(K_grid)} points | Maturités: {len(T_grid)}")

params_cm = HestonParams(
    kappa=torch.tensor(calib_params['kappa'], dtype=torch.float64),
    theta=torch.tensor(calib_params['theta'], dtype=torch.float64),
    sigma=torch.tensor(calib_params['sigma'], dtype=torch.float64),
    rho=torch.tensor(calib_params['rho'], dtype=torch.float64),
    v0=torch.tensor(calib_params['v0'], dtype=torch.float64),
)

call_prices_cm = np.zeros((len(T_grid), len(K_grid)))
put_prices_cm = np.zeros_like(call_prices_cm)
Ks_t = torch.tensor(K_grid, dtype=torch.float64)

for i, T_val in enumerate(T_grid):
    call_anal = carr_madan_call_torch(S0_ref, rf_rate, div_yield, float(T_val), params_cm, Ks_t)
    discount_factor = torch.exp(-torch.tensor(rf_rate * T_val, dtype=torch.float64))
    forward_factor = torch.exp(-torch.tensor(div_yield * T_val, dtype=torch.float64))
    put_anal = call_anal - S0_ref * forward_factor + Ks_t * discount_factor
    call_prices_cm[i, :] = call_anal.detach().cpu().numpy()
    put_prices_cm[i, :] = put_anal.detach().cpu().numpy()

print("Prix Carr-Madan calculés.")

call_iv_cm = np.zeros_like(call_prices_cm)
for i, T_val in enumerate(T_grid):
    for j, K_val in enumerate(K_grid):
        call_iv_cm[i, j] = implied_vol_option(call_prices_cm[i, j], S0_ref, K_val, T_val, rf_rate, "call")

KK_cm, TT_cm = np.meshgrid(K_grid, T_grid)
fig_iv_calls_cm = go.Figure(data=[go.Surface(x=KK_cm, y=TT_cm, z=call_iv_cm, colorscale='Viridis')])
fig_iv_calls_cm.update_layout(title=f"IV Surface Calls BS (Carr-Madan) - {ticker}", scene=dict(xaxis_title='Strike K', yaxis_title='Maturité (années)', zaxis_title='IV'), height=600)
fig_iv_calls_cm.show()

market_iv_df = calls_df.dropna(subset=["iv_market"]).copy()
market_iv_df = market_iv_df[(market_iv_df["T"] >= MIN_IV_MATURITY) & (market_iv_df["iv_market"] > 0)]
if len(market_iv_df) >= 5:
    points = market_iv_df[["K", "T"]].to_numpy()
    values = market_iv_df["iv_market"].to_numpy()
    market_iv_surface = griddata(points, values, (KK_cm, TT_cm), method="linear")
    if market_iv_surface is None or np.all(np.isnan(market_iv_surface)):
        market_iv_surface = griddata(points, values, (KK_cm, TT_cm), method="nearest")
    else:
        nan_mask = np.isnan(market_iv_surface)
        if nan_mask.any():
            market_iv_surface[nan_mask] = griddata(points, values, (KK_cm[nan_mask], TT_cm[nan_mask]), method="nearest")
    fig_iv_market = go.Figure(data=[go.Surface(x=KK_cm, y=TT_cm, z=market_iv_surface, colorscale='Plasma')])
    fig_iv_market.update_layout(title=f"IV Surface Calls (Marché) - {ticker}", scene=dict(xaxis_title='Strike K', yaxis_title='Maturité (années)', zaxis_title='IV'), height=600)
    fig_iv_market.show()
else:
    print("Pas assez de points IV marché pour construire une surface.")

Grille strikes: 41 points | Maturités: 40
Prix Carr-Madan calculés.


## Étape 4 — Pricing Monte Carlo Heston

In [75]:
n_points_mc = int(2 * span_mc / step_strike) + 1
S_grid_mc = np.linspace(S0_ref - span_mc, S0_ref + span_mc, n_points_mc)
K_grid_mc = np.linspace(S0_ref - span_mc, S0_ref + span_mc, n_points_mc)
n_steps_mc = 252#max(int(T_mc * 365 * 2), 252)

print(f"Grille MC: {len(S_grid_mc)} x {len(K_grid_mc)} (T = {T_mc:.2f} ans, n_steps = {n_steps_mc})")

call_prices_mc = np.zeros((len(S_grid_mc), len(K_grid_mc)))
put_prices_mc = np.zeros_like(call_prices_mc)

for i, S_val in enumerate(S_grid_mc):
    for j, K_val in enumerate(K_grid_mc):
        call_prices_mc[i, j] = heston_mc_pricer(
            S_val,
            K_val,
            T_mc,
            rf_rate,
            calib_params['v0'],
            calib_params['theta'],
            calib_params['kappa'],
            calib_params['sigma'],
            calib_params['rho'],
            n_paths=n_paths,
            n_steps=n_steps_mc,
            option_type="call",
        )
        put_prices_mc[i, j] = heston_mc_pricer(
            S_val,
            K_val,
            T_mc,
            rf_rate,
            calib_params['v0'],
            calib_params['theta'],
            calib_params['kappa'],
            calib_params['sigma'],
            calib_params['rho'],
            n_paths=n_paths,
            n_steps=n_steps_mc,
            option_type="put",
        )

fig_call_mc = go.Figure(data=go.Heatmap(z=call_prices_mc, x=K_grid_mc, y=S_grid_mc, colorscale='Viridis', colorbar=dict(title='Call MC')))
fig_call_mc.update_layout(title=f"Heatmap Prix Calls Heston (MC, T={T_mc:.2f}) - {ticker}", xaxis_title='Strike K', yaxis_title='Spot S', height=500)
fig_call_mc.show()

fig_put_mc = go.Figure(data=go.Heatmap(z=put_prices_mc, x=K_grid_mc, y=S_grid_mc, colorscale='Viridis', colorbar=dict(title='Put MC')))
fig_put_mc.update_layout(title=f"Heatmap Prix Puts Heston (MC, T={T_mc:.2f}) - {ticker}", xaxis_title='Strike K', yaxis_title='Spot S', height=500)
fig_put_mc.show()

Grille MC: 41 x 41 (T = 0.86 ans, n_steps = 252)


## Étape 5 — Comparaison Monte Carlo vs Carr-Madan

In [76]:
idx_S = len(S_grid_mc) // 2
S_compare = S_grid_mc[idx_S]
print(f"Spot sélectionné pour la comparaison: {S_compare:.2f}")

params_cm_compare = HestonParams(
    kappa=torch.tensor(calib_params['kappa'], dtype=torch.float64),
    theta=torch.tensor(calib_params['theta'], dtype=torch.float64),
    sigma=torch.tensor(calib_params['sigma'], dtype=torch.float64),
    rho=torch.tensor(calib_params['rho'], dtype=torch.float64),
    v0=torch.tensor(calib_params['v0'], dtype=torch.float64),
)

Ks_compare = torch.tensor(K_grid_mc, dtype=torch.float64)
call_anal_compare = carr_madan_call_torch(S_compare, rf_rate, div_yield, T_mc, params_cm_compare, Ks_compare)
discount_factor = torch.exp(-torch.tensor(rf_rate * T_mc, dtype=torch.float64))
forward_factor = torch.exp(-torch.tensor(div_yield * T_mc, dtype=torch.float64))
put_anal_compare = call_anal_compare - S_compare * forward_factor + Ks_compare * discount_factor

call_anal_np = call_anal_compare.detach().cpu().numpy()
put_anal_np = put_anal_compare.detach().cpu().numpy()

# Points marché à superposer (T cible et fenêtre K définie)
market_T_target = 0.86
k_min_plot, k_max_plot = S0_ref - span_mc, S0_ref + span_mc

market_calls_plot = calls_df.copy() if "calls_df" in globals() else calls_df_full.copy()
market_calls_plot = market_calls_plot.dropna(subset=["iv_market"])
market_calls_plot = market_calls_plot[(market_calls_plot["iv_market"] > 0) & (market_calls_plot["T"].round(2) == round(market_T_target, 2))]
market_calls_plot = market_calls_plot[(market_calls_plot["K"] >= k_min_plot) & (market_calls_plot["K"] <= k_max_plot)]
market_calls_plot = market_calls_plot.sort_values(["T", "K"])

market_puts_plot = puts_df.copy() if "puts_df" in globals() else puts_df_full.copy()
market_puts_plot = market_puts_plot.dropna(subset=["iv_market"])
market_puts_plot = market_puts_plot[(market_puts_plot["iv_market"] > 0) & (market_puts_plot["T"].round(2) == round(market_T_target, 2))]
market_puts_plot = market_puts_plot[(market_puts_plot["K"] >= k_min_plot) & (market_puts_plot["K"] <= k_max_plot)]
market_puts_plot = market_puts_plot.sort_values(["T", "K"])

fig_compare = go.Figure()
fig_compare.add_trace(go.Scatter(x=K_grid_mc, y=call_prices_mc[idx_S, :], mode='lines+markers', name='MC Call', line=dict(color='red')))
fig_compare.add_trace(go.Scatter(x=K_grid_mc, y=call_anal_np, mode='lines', name='Carr-Madan Call', line=dict(color='red', dash='dash')))
fig_compare.add_trace(go.Scatter(x=K_grid_mc, y=put_prices_mc[idx_S, :], mode='lines+markers', name='MC Put', line=dict(color='green')))
fig_compare.add_trace(go.Scatter(x=K_grid_mc, y=put_anal_np, mode='lines', name='Carr-Madan Put', line=dict(color='green', dash='dash')))
if len(market_calls_plot):
    fig_compare.add_trace(go.Scatter(x=market_calls_plot['K'], y=market_calls_plot.get('C_mkt', market_calls_plot['price'] if 'price' in market_calls_plot else market_calls_plot['iv_market']), mode='markers', name='Marché Call (T≈0.86)', marker=dict(color='black', size=7, symbol='x'), text=market_calls_plot['T']))
if len(market_puts_plot):
    fig_compare.add_trace(go.Scatter(x=market_puts_plot['K'], y=market_puts_plot.get('P_mkt', market_puts_plot['price'] if 'price' in market_puts_plot else market_puts_plot['iv_market']), mode='markers', name='Marché Put (T≈0.86)', marker=dict(color='purple', size=7, symbol='cross'), text=market_puts_plot['T']))
fig_compare.update_layout(title=f"Comparaison MC vs Analytique (S={S_compare:.2f}, T={T_mc:.2f})", xaxis_title='Strike K', yaxis_title='Prix', height=500, yaxis=dict(range=[0, 250]))
fig_compare.show()

# Tableaux marché filtrés
market_calls = market_calls_plot.sort_values(["T", "K"])[["K", "T", "C_mkt", "iv_market"]]
market_puts = market_puts_plot.sort_values(["T", "K"])[["K", "T", "P_mkt", "iv_market"]]
print(f"Lignes marché CALL (T≈{market_T_target}, K in [{k_min_plot},{k_max_plot}]): {len(market_calls)}")
if len(market_calls):
    display(market_calls.head(50))
else:
    print("Aucune ligne marché (calls) pour ce filtre")

print(f"Lignes marché PUT (T≈{market_T_target}, K in [{k_min_plot},{k_max_plot}]): {len(market_puts)}")
if len(market_puts):
    display(market_puts.head(50))
else:
    print("Aucune ligne marché (puts) pour ce filtre")


Spot sélectionné pour la comparaison: 672.97


Lignes marché CALL (T≈0.86, K in [652.97,692.97]): 39


,K,T,C_mkt,iv_market
3439,653.0,0.86,62.77,0.2002
3440,654.0,0.86,62.09,0.1996
3441,655.0,0.86,61.42,0.1990
3442,656.0,0.86,60.75,0.1984
3443,657.0,0.86,60.09,0.1978
3444,658.0,0.86,59.42,0.1972
3445,659.0,0.86,58.76,0.1966
3446,660.0,0.86,58.11,0.1960
3447,661.0,0.86,57.45,0.1954
3448,662.0,0.86,56.80,0.1948


Lignes marché PUT (T≈0.86, K in [652.97,692.97]): 39


,K,T,P_mkt,iv_market
3442,653.0,0.86,36.80,0.2010
3443,654.0,0.86,37.12,0.2004
3444,655.0,0.86,37.45,0.1998
3445,656.0,0.86,37.77,0.1992
3446,657.0,0.86,38.09,0.1986
3447,658.0,0.86,38.42,0.1979
3448,659.0,0.86,38.75,0.1974
3449,660.0,0.86,39.09,0.1967
3450,661.0,0.86,39.42,0.1961
3451,662.0,0.86,39.77,0.1955
